# Building my own neural network

## install dependencies

In [1]:
!pip install scikit-learn

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from sklearn.preprocessing import  StandardScaler

## Import the dataset

In [3]:
data=pd.read_csv('diabetes.csv')
data.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


In [4]:
data.shape

(768, 8)

In [5]:
x=data.iloc[:, 0:-1].values
y_str=list(data.iloc[:, -1])

In [6]:
x

array([[  6. , 148. ,  72. , ...,   0. ,  33.6,  50. ],
       [  1. ,  85. ,  66. , ...,   0. ,  26.6,  31. ],
       [  8. , 183. ,  64. , ...,   0. ,  23.3,  32. ],
       ...,
       [  5. , 121. ,  72. , ..., 112. ,  26.2,  30. ],
       [  1. , 126. ,  60. , ...,   0. ,  30.1,  47. ],
       [  1. ,  93. ,  70. , ...,   0. ,  30.4,  23. ]])

In [ ]:
y_str

## Preprocessing the data

In [8]:
# convert the string to values
y_int=[]
for label in y_str:
  if(label=='positive'):
    y_int.append(1)
  else: y_int.append(0)

In [ ]:
# as the x is a float values so y will also be a float value
y=np.array(y_int, dtype='float64')
y

In [ ]:
sc=StandardScaler()
x=sc.fit_transform(x)
x

In [11]:
x=torch.tensor(x)
y=torch.tensor(y).unsqueeze(1)

In [12]:
print(x.shape, y.shape)

torch.Size([768, 7]) torch.Size([768, 1])


In [13]:
y.shape

torch.Size([768, 1])

In [14]:
class Dataset(Dataset):
  def __init__(self, x, y):
    self.x=x
    self.y=y

  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return len(self.x)


In [15]:
dataset=Dataset(x,y)


In [16]:
len(dataset)

768

In [17]:
# loading the data
trainloader=torch.utils.data.DataLoader(dataset=dataset,
                                        shuffle=True,
                                        batch_size=32)


## Building the neural network

In [18]:
class MyNetwork(nn.Module):
  def __init__(self, input_layers, output_layers):
    super(MyNetwork, self).__init__()
    self.fc1=nn.Linear(input_layers, 5)
    self.fc2=nn.Linear(5,4)
    self.fc3=nn.Linear(4, output_layers)
    self.tanh=nn.Tanh()
    self.sigmoid=nn.Sigmoid()

  def forward(self, x):
    x=self.fc1(x)
    x=self.tanh(x)
    x=self.fc2(x)
    x=self.tanh(x)
    x=self.fc3(x)
    out=self.sigmoid(x)
    return out.float()



In [19]:
model=MyNetwork(7, 1)
# loss function
criterion=torch.nn.BCELoss()
#optimizer
optimizer=torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

### Training the network

In [46]:
epoch=200

losses = []
accuracies = []
for e in range(epoch):
  running_loss=0
  for x, y in trainloader:
    x=x.float()
    y=y.float()
    out=model.forward(x)
    optimizer.zero_grad()
    loss=criterion( out.double(),y.double())
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()

  losses.append(running_loss)
  output=(out>0.5).float()
  accuracy=(output==y).float().mean()
  accuracies.append(accuracy.item())
  print(f'Epoch: {e+1}, Loss: {running_loss:.3f}, Accuracy: {accuracy.item(): .3f}')



Epoch: 1, Loss: 9.492, Accuracy:  0.875
Epoch: 2, Loss: 9.493, Accuracy:  0.750
Epoch: 3, Loss: 9.491, Accuracy:  0.750
Epoch: 4, Loss: 9.493, Accuracy:  0.750
Epoch: 5, Loss: 9.487, Accuracy:  0.812
Epoch: 6, Loss: 9.491, Accuracy:  0.781
Epoch: 7, Loss: 9.493, Accuracy:  0.781
Epoch: 8, Loss: 9.493, Accuracy:  0.938
Epoch: 9, Loss: 9.491, Accuracy:  0.875
Epoch: 10, Loss: 9.493, Accuracy:  0.812
Epoch: 11, Loss: 9.491, Accuracy:  0.781
Epoch: 12, Loss: 9.487, Accuracy:  0.688
Epoch: 13, Loss: 9.487, Accuracy:  0.875
Epoch: 14, Loss: 9.490, Accuracy:  0.875
Epoch: 15, Loss: 9.490, Accuracy:  0.781
Epoch: 16, Loss: 9.488, Accuracy:  0.906
Epoch: 17, Loss: 9.490, Accuracy:  0.781
Epoch: 18, Loss: 9.489, Accuracy:  0.875
Epoch: 19, Loss: 9.484, Accuracy:  0.750
Epoch: 20, Loss: 9.485, Accuracy:  0.938
Epoch: 21, Loss: 9.484, Accuracy:  0.781
Epoch: 22, Loss: 9.489, Accuracy:  0.781
Epoch: 23, Loss: 9.489, Accuracy:  0.719
Epoch: 24, Loss: 9.486, Accuracy:  0.906
Epoch: 25, Loss: 9.487, A

In [47]:
import matplotlib.pyplot as plt
print(accuracies)

[0.875, 0.75, 0.75, 0.75, 0.8125, 0.78125, 0.78125, 0.9375, 0.875, 0.8125, 0.78125, 0.6875, 0.875, 0.875, 0.78125, 0.90625, 0.78125, 0.875, 0.75, 0.9375, 0.78125, 0.78125, 0.71875, 0.90625, 0.84375, 0.90625, 0.875, 0.84375, 0.9375, 0.75, 0.96875, 0.8125, 0.90625, 0.84375, 0.90625, 0.8125, 0.8125, 0.9375, 0.75, 0.71875, 0.96875, 0.875, 0.84375, 0.90625, 0.6875, 0.9375, 0.84375, 0.84375, 0.75, 0.8125, 0.84375, 0.84375, 0.84375, 0.9375, 0.84375, 0.84375, 0.8125, 0.75, 0.875, 0.84375, 0.875, 0.8125, 0.90625, 0.8125, 0.78125, 0.78125, 0.78125, 0.84375, 0.90625, 0.84375, 0.8125, 0.78125, 0.8125, 0.6875, 0.875, 0.8125, 0.90625, 0.84375, 0.875, 0.8125, 0.84375, 0.8125, 0.75, 0.90625, 0.9375, 0.6875, 0.84375, 0.96875, 0.875, 0.90625, 0.8125, 0.875, 0.875, 0.875, 0.875, 0.9375, 0.78125, 0.75, 0.875, 0.8125, 0.78125, 0.9375, 0.75, 0.875, 0.6875, 0.8125, 0.8125, 0.8125, 0.8125, 0.8125, 0.75, 0.71875, 0.84375, 0.78125, 0.78125, 0.65625, 0.78125, 0.71875, 0.6875, 0.75, 0.9375, 0.75, 0.71875, 0.8125,

In [ ]:
# Plotting the learning curve
plt.figure(figsize=(10, 5))

# Plotting the loss
plt.subplot(1, 2, 1)
plt.plot(losses, label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plotting the accuracy
plt.subplot(1, 2, 2)
plt.plot(accuracies, label='Training Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()